# MACS 40200 Problem Set #4 (Kei Irizawa)

In [1]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
# This next command is specifically for Jupyter Notebook
%matplotlib notebook

import requests
from IPython.display import Image

# Question 1 (a)

Data on ($c_t, k_t, w_t, r_t$) are given. We will use the following set of characterizing equations of the Brock and Mirman model for estimation: 

\begin{equation} \tag{1}
(c_t)^{-1} -\beta E[r_{t+1}(c_{t+1})^{-1}] = 0  
\end{equation}

\begin{equation} \tag{2}
c_t+ k_{t+1} - w_t - r_tk_t = 0 \quad 
\end{equation}

\begin{equation} \tag{3}
w_t-(1-\alpha)(e)^{z_t}(k_t)^\alpha = 0 \quad 
\end{equation}

\begin{equation} \tag{4}
r_t - \alpha (e)^{z_t}(k_t)^{\alpha-1} =0 \quad 
\end{equation}

\begin{equation} \tag{5}
z_t = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t \\
\mbox{where }E[\epsilon_t] = 0 
\end{equation}

\begin{equation} \tag{6}
y_t = (e)^{z_t} (k_t)^\alpha
\end{equation}


### Data Structure

In [2]:
data = np.loadtxt('data/NewMacroSeries.txt', delimiter = ',')
df = pd.DataFrame(data, columns = ['ct', 'kt', 'wt', 'rt', 'yt'])
df.head()

,ct,kt,wt,rt,yt
0,1.128323e+07,8.040697e+06,1.120211e+07,1.008852,1.931398e+07
1,1.215464e+07,8.030754e+06,1.206726e+07,1.088112,2.080561e+07
2,1.097303e+07,8.650974e+06,1.089414e+07,0.911904,1.878300e+07
3,9.711635e+06,7.809971e+06,9.641815e+06,0.893986,1.662382e+07
4,9.245673e+06,6.912184e+06,9.179203e+06,0.961637,1.582621e+07


In [3]:
ct, kt, wt, rt, yt = np.loadtxt('data/NewMacroSeries.txt', unpack = True, delimiter = ",")

### Approach

One nice property of the Brock and Mirman model is that the households decision has a known analytical solution in which the optimal savings decision $k_{t+1}$ is a function of the productivity shock today $z_t$ and the amount of capital today $k_t$. 

\begin{equation} \tag{7}
k_{t+1} = \alpha \beta \epsilon ^ z_t k_t^\alpha
\end{equation}

With this solution and equations (2) through (5), we can simulate the data of the Brock and Mirman model given parameters ($\alpha, \beta, \rho, \mu, \sigma$). We will assume that $z_0 = \mu$ and $k_1 = $ mean($k_t$) from the data. These are initial values that will not change across simulations. Also assume that $\beta = 0.99$. Next we will draw a matrix of $S=1,000$ simulations (columns) of T = 100 (rows) from a uniform $u_{s,t}\stackrel{}{\sim}U(0,1)$ distribution. These draws will not change across this SMM estimation procedure. The process of simulations is described below:

(Step 1) For each guess of the parameter vector ($\alpha, \beta = 0.99, \rho, \mu, \sigma$), you can use $u_{s,t}$ to generate normally distributed errors $\epsilon_{s,t} \stackrel{}{\sim} N(0, \sigma)$ using the inverse cdf of the normal distribution, where s is the index of the simulation number (columns). 

(Step 2) With $\epsilon_{s,t}, \rho, \mu$, and $z_0 = \mu$ you can use (5) to generate the simulations for $z_{s,t}$.

(Step 3) With $\alpha, \beta, z_{s,t}$, and $k_1$, you can use (7) to generate simulated values for $k_{s,t+1}$.

(Step 4) With $\alpha, z_{s,t}$, and $k_{s,t}$, you can use (3) and (4) to generate simulated values for $w_{s,t}$ annd $r_{s,t}$ respectively. 

(Step 5) With $w_{s,t}, r_{s,t},$ and $k_{s,t}$, you can use (2) to generate simulated values for $c_{s,t}$.

(Step 6) With $\alpha, z_{s,t},$ and $k_{s,t}$, you can use (6) to generate simulated values for $y_{s,t}$.

In [4]:
def simulated_values(unif_vals, xvals, params):
    
    ct, kt, wt, rt, yt = xvals
    alpha, rho, mu, sigma = params
    
    N = unif_vals.shape[0]  # Number of periods
    S = unif_vals.shape[1]  # Number of simulations
    beta = 0.99
    z_0 = mu
    k_1 = np.mean(kt)
    
    ### (Step 1) normally distributed errors
    epsilon = sts.norm.ppf(unif_vals, loc = 0, scale = sigma)
    
    ### (Step 2) Simulated values for z_s,t
    z_sim = np.zeros((N, S))
    z_sim[0, :] = rho * z_0 + (1 - rho) * mu + epsilon[0, :]  # z_1
    for i in range(1, N):
        z_sim[i, :] = rho * z_sim[i - 1, :] + (1 - rho) * mu + \
                        epsilon[i, :]
    
    ### (Step 3) Simulated values for k_s,t and k_s,t+1 
    # We simulate for N+1 periods, so that we ensure k_s and k_s,t+1 have N periods
    k_sim_all = np.zeros((N + 1, S))  
    k_sim_all[0, :] = k_1
    for i in range(1, N + 1):
        k_sim_all[i, :] = alpha * beta * np.exp(z_sim[i - 1, :]) * \
                        k_sim_all[i - 1, :] ** alpha
    
    k_sim = k_sim_all[:-1, :]  # simulated values for k_s,t
    k1_sim = k_sim_all[1:, :]  # simulated values for k_s,t+1
        
    ### (Step 4) Simulated values for w_s,t and r_s,t 
    w_sim = np.zeros((N, S))
    for i in range(N):
        w_sim[i, :] = (1 - alpha) * np.exp(z_sim[i, :]) * \
                        k_sim[i, :] ** alpha
        
    r_sim = np.zeros((N, S))
    for i in range(N):
        r_sim[i, :] = alpha * np.exp(z_sim[i, :]) * (k_sim[i, :]) \
                        ** (alpha - 1)
    
    ### (Step 5) Simulated values for c_s,t
    c_sim = np.zeros((N, S))
    for i in range(N):
        c_sim[i, :] = w_sim[i, :] + r_sim[i, :] * k_sim[i, :] - k1_sim[i, :] 
    
    ### (Step 6) Simulated values for y_s,t
    y_sim = np.zeros((N, S))
    for i in range(N):
        y_sim[i, :] = np.exp(z_sim[i, :]) * (k_sim[i, :]) ** alpha
        
    return c_sim, k_sim, y_sim

### Moment Conditions

In [5]:
def data_moments(xvals):
    
    ct, kt, wt, rt, yt = xvals
    
    # data_moment 1: mean of ct
    dm1 = np.mean(ct)
    
    # data_moment 2: mean of kt
    dm2 = np.mean(kt)
    
    # data_moment 3: mean of ct / yt
    dm3 = np.mean(ct / yt)
    
    # data_moment 4: variance of yt
    dm4 = np.var(yt)
    
    # data_moment 5: correaltion between ct and c_(t-1)
    c_t = ct[1:]  # ct
    c_lag = ct[:-1]  # c_(t-1)
    dm5 = np.corrcoef(c_t, c_lag)[0][1]
    
    # data_moment 6: correlation betweeh ct and kt
    dm6 = np.corrcoef(ct, kt)[0][1]
    
    return dm1, dm2, dm3, dm4, dm5, dm6

In [6]:
def model_moments(c_sim, k_sim, y_sim):
    
    # model_moment 1: mean of ct
    mm1 = np.mean(c_sim, axis = 0)
    mm1 = np.mean(mm1)
    
    # model_moment 2: mean of kt
    mm2 = np.mean(k_sim, axis = 0)
    mm2 = np.mean(mm2)
    
    # model_moment 3: mean of ct / yt
    mm3 = np.mean((c_sim / y_sim), axis = 0)
    mm3 = np.mean(mm3)
    
    # model_moment 4: variance of yt  
    mm4 = np.var(y_sim, axis = 0)
    mm4 = np.mean(mm4)
    
    # model_moment 5: correlation betweeen ct and c_(t-1)
    mm5 = []
    for i in range(c_sim.shape[1]):
        mm5.append(np.corrcoef(c_sim[1:, i], c_sim[:-1, i])[0][1])  
    mm5 = np.mean(np.array(mm5))
    
    # model_moment 6: correlation between ct and kt 
    mm6 = []
    for i in range(c_sim.shape[1]):
        mm6.append(np.corrcoef(c_sim[:, i], k_sim[:, i])[0][1])
    mm6 = np.mean(np.array(mm6))
    
    return mm1, mm2, mm3, mm4, mm5, mm6

### Constructing Criterion Function for the Optimization Problem

In [7]:
def err_vec(xvals, unif_vals, params, simple):
    
    # data_moments:
    dm1, dm2, dm3, dm4, dm5, dm6 = data_moments(xvals)
    moms_data = np.array([[dm1], [dm2], [dm3], [dm4], [dm5], [dm6]])
    
    # model_moments:
    c_sim, k_sim, y_sim = simulated_values(unif_vals, xvals, params)
    mm1, mm2, mm3, mm4, mm5, mm6 = model_moments(c_sim, k_sim, y_sim)
    moms_model = np.array([[mm1], [mm2], [mm3], [mm4], [mm5], [mm6]])
    
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data

    return err_vec

In [8]:
def criterion(params, args):
    
    ct, kt, wt, rt, yt, unif_vals, W_hat = args
    
    err = err_vec((ct, kt, wt, rt, yt), unif_vals, params, simple = False)
    crit_val = err.T @ W_hat @ err 
    
    return crit_val

### SMM Optimization Problem 

In [9]:
# initial guess of parameters: used GMM estimated parameters from PS3 for alpha_init, rho_init, mu_init
alpha_init = 0.42410452
rho_init = 0.82875887
mu_init = 9.978308
sigma_init = 0.9
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])

np.random.seed(25)  
N = 100   # 100 periods
S = 1000  # 1000 simulations
unif_vals = sts.uniform.rvs(0, 1, size=(N, S))
W_hat_init = np.eye(6)
args_init = np.array([ct, kt, wt, rt, yt, unif_vals, W_hat_init])

bnds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))

results_SMM = opt.minimize(criterion, params_init, args = args_init, 
                           bounds = bnds, method = 'L-BFGS-B', tol = 1e-15)

alpha_SMM, rho_SMM, mu_SMM, sigma_SMM = results_SMM.x

In [10]:
results_SMM

      fun: array([[4.39206567e-06]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([7.47889477e-06, 4.92815542e-07, 4.10829360e-07, 1.28670640e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 575
      nit: 59
   status: 0
  success: True
        x: array([0.42103733, 0.91775245, 9.92752469, 0.08896295])

### SMM Estimates, Vector of Moment Differences at Optimum, Criterion Function Value 

In [11]:
print('Estimates:')
print('alpha_SMM=', alpha_SMM)
print('rho_SMM=', rho_SMM)
print('mu_SMM=', mu_SMM)
print('sigma_SMM=', sigma_SMM)
print('Vector of Moment Differences at Optimum =', 
      err_vec((ct, kt, wt, rt, yt), unif_vals, (alpha_SMM, rho_SMM, mu_SMM, sigma_SMM), simple = False))
print('Criterion Function =', results_SMM.fun)

Estimates:
alpha_SMM= 0.4210373343333368
rho_SMM= 0.9177524495375082
mu_SMM= 9.92752468923445
sigma_SMM= 0.08896294665425233
Vector of Moment Differences at Optimum = [[ 7.37244260e-04]
 [-7.44400895e-04]
 [-1.75789283e-03]
 [-1.61300312e-07]
 [ 3.20452648e-04]
 [-3.18632633e-04]]
Criterion Function = [[4.39206567e-06]]


### Computing the Standard Errors for Estimated Parameter vector 

In [12]:
def Jac_err(xvals, unif_vals, params, simple):
    
    ct, kt, wt, rt, yt = xvals
    alpha, rho, mu, sigma = params
    
    # 6 is the number of moment conditions, and 4 is the number of parameters we are estimating
    Jac_err = np.zeros((6,4))  
    h_alpha = 1e-4 * alpha
    h_rho = 1e-4 * rho
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    
    Jac_err[:, 0] = \
        ((err_vec(xvals, unif_vals, (alpha + h_alpha, rho, mu, sigma), simple) - 
          err_vec(xvals, unif_vals, (alpha - h_alpha, rho, mu, sigma), simple)) / (2 * h_alpha)).flatten()
    
    Jac_err[:, 1] = \
        ((err_vec(xvals, unif_vals, (alpha, rho + h_rho, mu, sigma), simple) - 
          err_vec(xvals, unif_vals, (alpha, rho - h_rho, mu, sigma), simple)) / (2 * h_rho)).flatten()
    
    Jac_err[:, 2] = \
        ((err_vec(xvals, unif_vals, (alpha, rho, mu + h_mu, sigma), simple) - 
          err_vec(xvals, unif_vals, (alpha, rho, mu - h_mu, sigma), simple)) / (2 * h_mu)).flatten()
    
    Jac_err[:, 3] = \
        ((err_vec(xvals, unif_vals, (alpha, rho, mu, sigma  + h_sigma), simple) - 
          err_vec(xvals, unif_vals, (alpha, rho, mu, sigma  - h_sigma), simple)) / (2 * h_sigma)).flatten()
    
    return Jac_err

In [13]:
S = unif_vals.shape[1]
xvals = (ct, kt, wt, rt, yt)
d_err = Jac_err(xvals, unif_vals, (alpha_SMM, rho_SMM, mu_SMM, sigma_SMM), simple = False)
print('Jacobian Matrix = ', d_err)
print('Initial Optimal Weighting Matrix =', W_hat_init)
SigHat = (1 / S) * lin.inv(d_err.T @ W_hat_init @ d_err)
print('Variance-Covariance Matrix =', SigHat)

Jacobian Matrix =  [[ 2.70595979e+01  8.52841212e-01  1.71640193e+00  1.56427193e+00]
 [ 3.07758291e+01  8.41574827e-01  1.69658829e+00  1.54610624e+00]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  1.06881049e-11]
 [ 6.13659525e+01  1.21935651e+01  3.45960323e+00  2.76849452e+01]
 [ 1.45237215e-01  4.25852739e-01  1.01858746e-03 -8.89506489e-02]
 [ 2.48230100e-01  4.30866376e-01  6.64600218e-03 -8.60840701e-02]]
Initial Optimal Weighting Matrix = [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
Variance-Covariance Matrix = [[ 9.02319947e-05 -2.69684872e-05 -1.51626295e-03  1.34877525e-06]
 [-2.69684872e-05  2.31752764e-03  1.98148948e-04 -9.85428065e-04]
 [-1.51626295e-03  1.98148948e-04  2.57275350e-02  5.72752807e-05]
 [ 1.34877525e-06 -9.85428065e-04  5.72752807e-05  4.25224643e-04]]


### Standard Errors for Estimated Parameter vector 

In [14]:
print('Std. err. alpha_hat =', np.sqrt(SigHat[0, 0]))
print('Std. err. rho_hat =', np.sqrt(SigHat[1, 1]))
print('Std. err. mu_hat =', np.sqrt(SigHat[2, 2]))
print('Std. err. sigma_hat =', np.sqrt(SigHat[3, 3]))

Std. err. alpha_hat = 0.009499052305994385
Std. err. rho_hat = 0.04814070670742166
Std. err. mu_hat = 0.16039805179809616
Std. err. sigma_hat = 0.02062097580744679


# Question 1 (b)

### Construction of the $E(\tilde{x},x|\theta)$ matrix

This $E(\tilde{x},x|\theta)$ matrix represents the contribution of the $s$th simulated moment to the $r$th moment error. Define $E(\tilde{x},x|\theta)$ as the $R\times S$ matrix of moment error functions from each simulation. When the errors are percent deviations, the $E(\tilde{x},x|\theta)$ matrix is the following.

$$ E(\tilde{x},x|\theta) =
  \begin{bmatrix}
    \frac{m_1(\tilde{x}_1|\theta) - m_1(x)}{m_1(x)} & \frac{m_1(\tilde{x}_2|\theta) - m_1(x)}{m_1(x)} & ... & \frac{m_1(\tilde{x}_S|\theta) - m_1(x)}{m_1(x)} \\
    \frac{m_2(\tilde{x}_1|\theta) - m_2(x)}{m_2(x)} & \frac{m_2(\tilde{x}_2|\theta) - m_2(x)}{m_2(x)} & ... & \frac{m_2(\tilde{x}_S|\theta) - m_2(x)}{m_2(x)} \\
    \vdots & \vdots & \ddots & \vdots \\
    \frac{m_R(\tilde{x}_1|\theta) - m_R(x)}{m_R(x)} & \frac{m_R(\tilde{x}_2|\theta) - m_R(x)}{m_R(x)} & ... & \frac{m_R(\tilde{x}_S|\theta) - m_R(x)}{m_R(x)} \\
  \end{bmatrix} $$

where $m_r(x)$ is the $r$th data moment which is constant across each row, and $m_r(\tilde{x}_s|\theta)$ is the $r$th model moment from the $s$th simulation which are changing across each row. 

In [15]:
def model_moments2(c_sim, k_sim, y_sim): 
    '''
    This function is different from model_moments previously defined.
    Constructs model moment vectors of length S. 
    Each element in the vector represents the model moment from simulation i.
    '''
    # model_moment 1 vector: mean of ct
    mm1 = np.mean(c_sim, axis = 0)
    
    # model_moment 2 vector: mean of kt
    mm2 = np.mean(k_sim, axis = 0)
    
    # model_moment 3 vector: mean of ct / yt
    mm3 = np.mean((c_sim / y_sim), axis = 0)
    
    # model_moment 4 vector: variance of yt  
    mm4 = np.var(y_sim, axis = 0)
    
    # model_moment 5 vector: correlation betweeen ct and c_(t-1)
    mm5 = []
    for i in range(c_sim.shape[1]):
        mm5.append(np.corrcoef(c_sim[1:, i], c_sim[:-1, i])[0][1])  
    mm5 = np.array(mm5)
    
    # model_moment 6 vector: correlation between ct and kt 
    mm6 = []
    for i in range(c_sim.shape[1]):
        mm6.append(np.corrcoef(c_sim[:, i], k_sim[:, i])[0][1])
    mm6 = np.array(mm6)
    
    return mm1, mm2, mm3, mm4, mm5, mm6

In [16]:
def get_Err_mat(xvals, params, simple):
    
    R = 6  # Number of moment conditions
    S = unif_vals.shape[1]  # In our case, it is 1000 simulations
    Err_mat = np.zeros((R,S))
    
    # data_moments:
    dm1, dm2, dm3, dm4, dm5, dm6 = data_moments(xvals)
    moms_data = np.array([[dm1], [dm2], [dm3], [dm4], [dm5], [dm6]])
    
    # model_moments:
    c_sim, k_sim, y_sim = simulated_values(unif_vals, xvals, params)
    mm1, mm2, mm3, mm4, mm5, mm6 = model_moments2(c_sim, k_sim, y_sim)
    moms_model = np.array([[mm1], [mm2], [mm3], [mm4], [mm5], [mm6]])
    
    if simple:
        Err_mat[0, :] = mm1 - dm1
        Err_mat[1, :] = mm2 - dm2
        Err_mat[2, :] = mm3 - dm3
        Err_mat[3, :] = mm4 - dm4
        Err_mat[4, :] = mm5 - dm5
        Err_mat[5, :] = mm6 - dm6
        
    else:
        Err_mat[0, :] = (mm1 - dm1) / dm1
        Err_mat[1, :] = (mm2 - dm2) / dm2
        Err_mat[2, :] = (mm3 - dm3) / dm3
        Err_mat[3, :] = (mm4 - dm4) / dm4
        Err_mat[4, :] = (mm5 - dm5) / dm5
        Err_mat[5, :] = (mm6 - dm6) / dm6
    
    return Err_mat

### Construction of the Optimal Weighting Matrix

We use the $E(\tilde{x},x|\theta)$ data matrix and the Step 1 SMM estimate $e(x|\hat{\theta}_{1,SMM})$ to get a new estimate of the variance covariance matrix.

$$ \hat{\Omega}_2 = \frac{1}{S}E(\tilde{x},x|\hat{\theta}_{1,SMM})\,E(\tilde{x},x|\hat{\theta}_{1,SMM})^T $$
  
This is simply saying that the $(r,s)$-element of the estimator of the variance-covariance matrix of the moment vector is the following.

$$\hat{\Omega}_{r,s} = \frac{1}{S}\sum_{i=1}^S\Bigl[m_r(\tilde{x}_i|\theta) - m_{r}(x)\Bigr]\Bigl[ m_s(\tilde{x}_i|\theta) - m_s(x)\Bigr] $$

The optimal weighting matrix is the inverse of the two-step variance covariance matrix.

$$ \hat{W}^{two-step} \equiv \hat{\Omega}_2^{-1} $$

In [17]:
xvals = (ct, kt, wt, rt, yt)
params = (alpha_SMM, rho_SMM, mu_SMM, sigma_SMM)
Err_mat = get_Err_mat(xvals, params, simple = False)
VCV2 = (1 / unif_vals.shape[1]) * (Err_mat @ Err_mat.T)
print('Variance-Covariance Matrix of Moment Vector =', VCV2)
W_hat2 = lin.inv(VCV2)
print('Optimal Two-Step Weighting Matrix =', W_hat2)

Variance-Covariance Matrix of Moment Vector = [[ 3.25711109e-02  3.23087994e-02 -1.29599640e-06  9.46589494e-02
   6.04896910e-04  6.42882650e-04]
 [ 3.23087994e-02  3.20636654e-02  1.30857699e-06  9.39284291e-02
   5.99351955e-04  6.38107909e-04]
 [-1.29599640e-06  1.30857699e-06  3.09018719e-06  2.83548663e-10
  -5.63321412e-07  5.60122021e-07]
 [ 9.46589494e-02  9.39284291e-02  2.83548663e-10  6.61004952e-01
   1.15166589e-02  1.16540804e-02]
 [ 6.04896910e-04  5.99351955e-04 -5.63321412e-07  1.15166589e-02
   6.74669774e-04  6.76761493e-04]
 [ 6.42882650e-04  6.38107909e-04  5.60122021e-07  1.16540804e-02
   6.76761493e-04  6.80312368e-04]]
Optimal Two-Step Weighting Matrix = [[ 7.64716417e+04 -7.70661699e+04  6.14061889e+04 -8.09344842e-01
  -9.10844929e+03  9.04514758e+03]
 [-7.70661699e+04  7.77273899e+04 -6.14758801e+04 -1.01325278e+01
   1.04406183e+04 -1.02412482e+04]
 [ 6.14061889e+04 -6.14758801e+04  5.04942902e+05 -4.05050706e+00
   3.56924662e+05 -3.55773659e+05]
 [-8.093

### Two-Step SMM Optimization Problem

We will re-estimate the SMM estimator using the optimal two-step weighting matrix.

$$ \hat{\theta}_{2,SMM}=\theta:\quad \min_{\theta}\:e(\tilde{x},x|\theta)^T \, \hat{W}^{two-step} \, e(\tilde{x},x|\theta) $$

$\hat{\theta}_{2, SMM}$ is called the two-step SMM estimator.

In [18]:
params_init2 = np.array([alpha_SMM, rho_SMM, mu_SMM, sigma_SMM])
args_init2 = np.array([ct, kt, wt, rt, yt, unif_vals, W_hat2])
bnds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))

results_SMM2 = opt.minimize(criterion, params_init2, args = (args_init2),
                         method = 'L-BFGS-B', bounds = bnds)

alpha_SMM2, rho_SMM2, mu_SMM2, sigma_SMM2 = results_SMM2.x

In [19]:
results_SMM2

      fun: array([[0.73844739]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.13197262e-03,  5.30464561e-05, -1.01363362e-05,  1.42408307e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 155
      nit: 27
   status: 0
  success: True
        x: array([0.42076554, 0.91776852, 9.92957699, 0.08869718])

### SMM Estimates, Vector of Moment Differences at Optimum, Criterion Function Value 

In [20]:
print('Estimates:')
print('alpha_SMM=', alpha_SMM2)
print('rho_SMM=', rho_SMM2)
print('mu_SMM=', mu_SMM2)
print('sigma_SMM=', sigma_SMM2)
print('Vector of Moment Differences at Optimum =', 
      err_vec((ct, kt, wt, rt, yt), unif_vals, (alpha_SMM2, rho_SMM2, mu_SMM2, sigma_SMM2), simple = False))
print('Criterion Function =', results_SMM2.fun)

Estimates:
alpha_SMM= 0.4207655355747233
rho_SMM= 0.9177685235374092
mu_SMM= 9.929576993457708
sigma_SMM= 0.08869718019244954
Vector of Moment Differences at Optimum = [[-0.0034851 ]
 [-0.00600784]
 [-0.0012973 ]
 [-0.01660245]
 [ 0.0003134 ]
 [-0.00034344]]
Criterion Function = [[0.73844739]]


### Computing the Standard Errors for Estimated Parameter vector 

In [21]:
S = unif_vals.shape[1]
xvals = (ct, kt, wt, rt, yt)
d_err2 = Jac_err(xvals, unif_vals, (alpha_SMM2, rho_SMM2, mu_SMM2, sigma_SMM2), False)
print('Jacobian Matrix = ', d_err2)
print('Optimal Two-Step Weighting Matrix =', W_hat2)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W_hat2 @ d_err2)
print('Variance-Covariance Matrix =', SigHat2)

Jacobian Matrix =  [[ 2.69229384e+01  8.43815279e-01  1.70834986e+00  1.55210711e+00]
 [ 3.05906178e+01  8.31742917e-01  1.68676140e+00  1.53238413e+00]
 [-1.69462513e+00  0.00000000e+00  9.56497043e-14  0.00000000e+00]
 [ 6.02918020e+01  1.19712805e+01  3.40025804e+00  2.72726955e+01]
 [ 1.46579608e-01  4.26155087e-01  1.09044904e-03 -8.86052985e-02]
 [ 2.57013179e-01  4.31274884e-01  7.12635238e-03 -8.53776892e-02]]
Optimal Two-Step Weighting Matrix = [[ 7.64716417e+04 -7.70661699e+04  6.14061889e+04 -8.09344842e-01
  -9.10844929e+03  9.04514758e+03]
 [-7.70661699e+04  7.77273899e+04 -6.14758801e+04 -1.01325278e+01
   1.04406183e+04 -1.02412482e+04]
 [ 6.14061889e+04 -6.14758801e+04  5.04942902e+05 -4.05050706e+00
   3.56924662e+05 -3.55773659e+05]
 [-8.09344842e-01 -1.01325278e+01 -4.05050706e+00  4.12824618e+00
  -5.22824324e+01 -8.43722291e+00]
 [-9.10844929e+03  1.04406183e+04  3.56924662e+05 -5.22824324e+01
   1.00676826e+06 -1.00209729e+06]
 [ 9.04514758e+03 -1.02412482e+04 -3.

### Standard Errors for Estimated Parameter vector 

In [22]:
print('Std. err. alpha_hat =', np.sqrt(SigHat2[0, 0]))
print('Std. err. rho_hat =', np.sqrt(SigHat2[1, 1]))
print('Std. err. mu_hat =', np.sqrt(SigHat2[2, 2]))
print('Std. err. sigma_hat =', np.sqrt(SigHat2[3, 3]))

Std. err. alpha_hat = 1.680579749813418e-05
Std. err. rho_hat = 0.0018568519647986026
Std. err. mu_hat = 0.0029433033982086375
Std. err. sigma_hat = 0.0006670432789219563
